In [ ]:
day_obs = "20241104" 

# Facilities Temperatures Reports
Author: Bruno C. Quint

This notebook contains a plot used to evaluate the safety of the M1M3 glass.
The temperature requirements are:
* The difference between the mirror cell and its surroundings must be below 5ºC. 
* The temperature increase/decrease rate must be less than 1ºC/h. 

The details about each topic are described below. 
Some of them might not be used in the current version.
However, I am leaving them here until this notebook gets mature and we reach an agreement on what needs monitoring. 


In [ ]:
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os
import pandas as pd
import warnings

from astropy import units as u
from astropy.time import Time
from scipy.interpolate import interp1d
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, getDayObsStartTime
from lsst.ts.xml.sal_enums import State


# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Constants used in the notebook
ess_weather_station_sal_index = 301
m1m3_inside_cell_sal_index = 113
dome_inside_sal_index = 111

# Create a folder for plots
os.makedirs("plots/", exist_ok=True)

## Query the data

In [ ]:
day_obs = int(day_obs)
start_time = getDayObsStartTime(day_obs)
end_time = start_time + 1 * u.day

print(
    f"\nQuery data for {day_obs}"
    f"\n  starts at {start_time.isot} and"
    f"\n  ends at {end_time.isot}\n"
)

The cell bellow contains data that comes from the Weather Station Tower.

In [ ]:
df_outside = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_outside.salIndex == ess_weather_station_sal_index
df_outside = df_outside[mask]

# Print the location of this sensor
print("Sensor location: ", df_outside["location"].unique())

# We do not need the salIndex anymore
df_outside = df_outside.drop(columns=['salIndex'])

# Get the rolling min/mean/max values for the temperature
df_outside = df_outside.rename(columns={"temperatureItem0": "temperature"})
df_outside = df_outside.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_outside.columns = df_outside.columns.droplevel(0)

MTMount contains multiple temperature sensors near the telescope.  
For the sake of simplicity, we will monitor the temperature sensors near the 
Top End Assembly for now.

In [ ]:
df_inside = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.topEndChiller",
    columns=["ambientTemperature"],
    begin=start_time,
    end=end_time,
)

# Get the rolling min/mean/max values for the temperature
df_inside = df_inside.rename(columns={"ambientTemperature": "temperature"})
df_inside = df_inside.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_inside.columns = df_inside.columns.droplevel(0)

The telemetry above has not been reliable.  
As an attempt of investigating it a bit more, we can visualize what are the summary states of MTMount. 

In [ ]:
df_mtmount = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.logevent_summaryState",
    columns=["summaryState"],
    begin=start_time,
    end=end_time,
)

if df_mtmount.index.empty:
    df_mtmount = pd.DataFrame(columns=["summaryState"])

The glycol cooling system is divided into three main circuits, each associated with different chillers and purposes. 
The system is designed with redundancy in mind, allowing automatic switching between chillers in case of failure.

- **Chiller 1 & 2**: Located on **Level 1** – Cold Glycol Circuit
- **Chiller 3**: Located on **Level 1** – General Purpose (GP) Glycol / Comfort Circuit
- **Chiller 4**: Located **outside the building on Level 1** – Dedicated Cooling Circuit


### Cold Glycol – Chillers 1 & 2

Chillers 1 and 2 provide the cold glycol circuit, operating at lower temperatures (–4°C to +4°C).
This circuit cools several systems:

- OSS
- Dome AHUs (level 5)
- M1M3TS
- Dyn Chiller 2 (TMA/Camera)
- Camera cabinet volumes (cryo and PCS)

In [ ]:
df_glycol_cold = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.cooling",
    columns=["glycolTemperaturePier0101"],
    begin=start_time,
    end=end_time,
)

df_glycol_cold = df_glycol_cold.rename(columns={"glycolTemperaturePier0101": "temperature"})
df_glycol_cold = df_glycol_cold.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_glycol_cold.columns = df_glycol_cold.columns.droplevel(0)

### General Glycol (Comfort) – Chiller 3

Chiller 3 handles the general purpose (GP) glycol, also called the comfort circuit, operating typically at 5–12°C.  
It was originally installed to cool:

- MRCR (camera)
- Office AC
- Cleanroom AHUs (level 3/4)
- Dyn Chiller 1 (level 3 cleanroom)
- TMA cabinets/drives
- camera compressors (cryo and PCS)


The system was extended to provide cooling to Levels 4, 6, and 7. However, Level 8 is not reached by this circuit.  



### Chiller 4

Chiller 4 supplies computer room AC or coating plant (switched via valve)


In [ ]:
df_glycol_general = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.generalPurposeGlycolWater",
    columns=["glycolTemperaturePier0001"],
    begin=start_time,
    end=end_time,
)

# Get the rolling min/mean/max values for the temperature
df_glycol_general = df_glycol_general.rename(columns={"glycolTemperaturePier0001": "temperature"})
df_glycol_general = df_glycol_general.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_glycol_general.columns = df_glycol_general.columns.droplevel(0)

The `df_m1m3` below represents the air above M1M3.  
We can use it as an estimate of the temperature inside the dome and near the cell. 

In [ ]:
df_m1m3 = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_m1m3.salIndex == m1m3_inside_cell_sal_index
df_m1m3 = df_m1m3[mask]

# Print the location of this sensor
print("Sensor location: ", df_m1m3["location"].unique())

# We do not need the salIndex anymore
df_m1m3 = df_m1m3.drop(columns=['salIndex'])

# Get the rolling min/mean/max values for the temperature
df_m1m3 = df_m1m3.rename(columns={"temperatureItem0": "temperature"})
df_m1m3 = df_m1m3.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_m1m3.columns = df_m1m3.columns.droplevel(0)

This is another measurement of the temperature inside the dome.  
We need more details about its location since the output is not clear. 

In [ ]:
df_inside_dome = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_inside_dome.salIndex == dome_inside_sal_index
df_inside_dome = df_inside_dome[mask]

# Print the location of this sensor
print("Sensor location: ", df_inside_dome["location"].unique())

# We do not need the salIndex anymore
df_inside_dome = df_inside_dome.drop(columns=['salIndex'])

# Get the rolling min/mean/max values for the temperature
df_inside_dome = df_inside_dome.rename(columns={"temperatureItem0": "temperature"})
df_inside_dome = df_inside_dome.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_inside_dome.columns = df_inside_dome.columns.droplevel(0)

The telemetry below retrieves the temperatures from inside the cell.  
There are 96 sensors in this topic.  
This is why the statistics are calculated across them intead of a rolling window. 

In [ ]:
df_m1m3ts_inside = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3TS.thermalData",
    columns=[f"absoluteTemperature{i}" for i in range(96)],
    begin=start_time,
    end=end_time
)

df_m1m3ts_inside["mean"] = df_m1m3ts_inside.mean(axis=1)
df_m1m3ts_inside["min"] = df_m1m3ts_inside.min(axis=1)
df_m1m3ts_inside["max"] = df_m1m3ts_inside.max(axis=1)

The mixing valve allows glycol circulation within the cell.  
It ranges from 0% to 100% opened.  
The more it is opened, the more it will cool down the m1m3 cell. 

In [ ]:
df_m1m3ts_mixing_valve = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3TS.mixingValve",
    columns=["valvePosition"],
    begin=start_time,
    end=end_time
)

This topic came from Petr and Sandrine as another estimate of the temperature inside the cell.  
The topic name and the documentation in the ts-xml page confirms that.  
It is not clear if it has any special spatial distribution that can be relevant in the analysis.

In [ ]:
df_m1m3ts_glycol_loop = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3TS.glycolLoopTemperature",
    columns=[f"insideCellTemperature{i}" for i in range(1, 4)],
    begin=start_time,
    end=end_time
)

df_m1m3ts_glycol_loop_1 = pd.DataFrame(df_m1m3ts_glycol_loop["insideCellTemperature1"])
df_m1m3ts_glycol_loop_1 = df_m1m3ts_glycol_loop_1.rename(columns={"insideCellTemperature1": "temperature"})
df_m1m3ts_glycol_loop_1 = df_m1m3ts_glycol_loop_1.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_m1m3ts_glycol_loop_1.columns = df_m1m3ts_glycol_loop_1.columns.droplevel(0)

df_m1m3ts_glycol_loop_2 = pd.DataFrame(df_m1m3ts_glycol_loop["insideCellTemperature2"])
df_m1m3ts_glycol_loop_2 = df_m1m3ts_glycol_loop_2.rename(columns={"insideCellTemperature2": "temperature"})
df_m1m3ts_glycol_loop_2 = df_m1m3ts_glycol_loop_2.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_m1m3ts_glycol_loop_2.columns = df_m1m3ts_glycol_loop_2.columns.droplevel(0)

df_m1m3ts_glycol_loop_3 = pd.DataFrame(df_m1m3ts_glycol_loop["insideCellTemperature3"])
df_m1m3ts_glycol_loop_3 = df_m1m3ts_glycol_loop_3.rename(columns={"insideCellTemperature3": "temperature"})
df_m1m3ts_glycol_loop_3 = df_m1m3ts_glycol_loop_3.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_m1m3ts_glycol_loop_3.columns = df_m1m3ts_glycol_loop_3.columns.droplevel(0)

In [ ]:
df_m2 = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time
)

# Select the data from the weather station using the salIndex
mask = df_m2.salIndex == 112
df_m2 = df_m2[mask]

# Print the location of this sensor
print("Sensor location: ", df_m2["location"].unique())

# We do not need the salIndex anymore
df_m2 = df_m2.drop(columns=['salIndex'])

# Get the rolling min/mean/max values for the temperature
df_m2 = df_m2.rename(columns={"temperatureItem0": "temperature"})
df_m2 = df_m2.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_m2.columns = df_m2.columns.droplevel(0)

In [ ]:
count = 336
fields = ["timestamp", "temperature", "temperatureSpread"]

df_weather_forecast = getEfdData(
    client=client,
    topic="lsst.sal.WeatherForecast.hourlyTrend",
    columns=[f"{field}{i}" for field, i in itertools.product(fields, range(count))],
    begin=start_time,
    end=end_time
)

# This will be published near 23:34 UTC
s_weather_forecast = df_weather_forecast.iloc[0]

weather_forecast_update_date = s_weather_forecast.name

timestamps = Time(s_weather_forecast[[f"timestamp{i}" for i in range(count)]], format="unix", scale="utc").isot
timestamps = pd.to_datetime(timestamps)
temperatures = s_weather_forecast[[f"temperature{i}" for i in range(count)]]
temperatures_spread = s_weather_forecast[[f"temperatureSpread{i}" for i in range(count)]]

df_weather_forecast = pd.DataFrame(
    np.array([temperatures, temperatures_spread]).T,
    columns=["temperature", "temperatureSpread"],
    index=timestamps
)

# # df = df_weather_forecast.set_index("timestamp")
# df = df[(df.index >= start_time) & (df.index <= end_time)]
# df["temperature"] = pd.to_numeric(df["temperature"])
# df["temperatureSpread"] = pd.to_numeric(df["temperatureSpread"])

# df["mean"] = df["temperature"]
# df["min"] = df["temperature"] - 0.5 * np.array(df["temperatureSpread"])
# df["max"] = df["temperature"] + 0.5 * np.array(df["temperatureSpread"])

# df_weather_forecast = df

In [ ]:
start_time.datetime
#df_weather_forecast.index

## Temperatures Plots 

For each plot we have a solid line representing the rolling average per minute. 
In addition, the figure below contains a shaded region per color showing the min/max 
values per telemetry. However, since the min/max values do not stray far from the 
average, they are almost imperceptible.

In [ ]:
def plot_temperature(ax, df, label, color, alpha=0.5):
    """
    Add a new plot to the axes with the mean temperature
    and the min/max fill_between.

    Parameters
    ----------
    ax : matplotlib.pyplot.Axes
        Axes that will hold the plot.
    df : pandas.DataFrame
        Dataframe containing the `mean`, `min`, `max` columns.
    label : str
        A string to add to the legend.
    color : str
        A string representing the color of the plots.
    alpha : float
        A float representing the transparency of the fill_between.
    """
    ax.plot(
        df.index,
        df["mean"],
        label=label,
        color=color
    )
    ax.fill_between(
        df.index,
        df["min"],
        df["max"],
        color=color,
        alpha=alpha
    )

In [ ]:
def evaluate_data(m1m3_temp_df, outside_temp_df):
    """
    Makes some calculations on the temperature data, evaluating if 
    temperature difference between M1M3 and outside temperature and 
    the rate of change.

    Also evaluates the Risk:
    low(0): temp_diff < 4 C OR hour_time_change <0.7 deg/hour
    warning(1): 4 < temp_diff < 5 C OR 0.7 < hour_time_change <1 deg/hour
    critical(2): temp_diff > 5 OR hour_time_change > 1 deg/hour

    Parameters
    ----------
    m1m3_temp_df : pandas.DataFrame
        The data you want to use that signifies the M1M3 data
    outside_temp_df : pandas.DataFrame
        The data you want to use that signifies the Outside temperature

    Returns
    -------
    m1m3_temp_df.index : list
        Time to plot against the Temperature Difference and Hourly Rate of Change
    temp_diff : list 
        Temperature Difference between M1M3 and the outside temperature
    hour_time_change : list
        Hourly Rate of Change of M1M3
    risk : int
        Risk Level as defined above
    """
    # Interpolate data streams to determine temperature difference
    m1m3_time = m1m3_temp_df.index.astype('int') / 1e-9
    outside_time = outside_temp_df.index.astype('int') / 1e-9
    m1m3_temp = interp1d(m1m3_time, m1m3_temp_df['mean'], bounds_error=False, fill_value="extrapolate")
    outside_temp = interp1d(outside_time, outside_temp_df['mean'], bounds_error=False, fill_value='extrapolate')
    temp_diff = m1m3_temp(m1m3_time) - outside_temp(m1m3_time)

    # Calculate the hourly rate of change
    total_time = m1m3_temp_df.index[-1] - m1m3_temp_df.index[0]
    spacing = total_time/len(m1m3_temp_df)
    period = pd.Timedelta(hours=1)/spacing
    prev_temp = m1m3_temp_df.shift(periods=int(period))
    hour_time_change = prev_temp['mean']-m1m3_temp_df['mean']

    # Evaluate the Risk
    if temp_diff[-1] < 4 :
        risk = 0
    elif (temp_diff[-1] > 4) | (temp_diff[-1] < 5):
        risk = 1
    elif temp_diff[-1] > 5:
        risk = 2

    if hour_time_change[-1] < 0.7:
        risk2 = 0
    elif (hour_time_change[-1] > 0.7) & (hour_time_change[-1] < 1):
        risk2 = 1
    elif hour_time_change[-1] > 1:
        risk2 = 2
    return m1m3_temp_df.index, list(temp_diff), list(hour_time_change), max(risk, risk2)
time_plot, temp_diff, rate_change, risk = evaluate_data(df_m1m3, df_outside)

In [ ]:
#%matplotlib widget

fig = plt.figure(figsize=(15,15), num=f"temperatures_{day_obs}")
gs = fig.add_gridspec(2, 1, hspace=0.6, height_ratios=[6,4])
ax = fig.add_subplot(gs[0])
plot_temperature(ax, df_m1m3ts_inside, label="M1M3TS Thermal Data", color="goldenrod", alpha=0.15)
plot_temperature(ax, df_m1m3ts_glycol_loop_1, label="M1M3TS Glycol Loop 1 Temp", color="firebrick")
plot_temperature(ax, df_m1m3ts_glycol_loop_2, label="M1M3TS Glycol Loop 2 Temp", color="orangered")
plot_temperature(ax, df_m1m3ts_glycol_loop_3, label="M1M3TS Glycol Loop 3 Temp", color="tomato")
plot_temperature(ax, df_glycol_cold, label="Glycol Cold Temp", color="blueviolet")
plot_temperature(ax, df_glycol_general, label="Glycol General Temp", color="darkmagenta")
plot_temperature(ax, df_m1m3, label="M1M3 Air Temp", color="darkturquoise")
plot_temperature(ax, df_inside_dome, label="Camera Temp", color="lightseagreen")
plot_temperature(ax, df_m2, label="M2 Temp", color="teal")
plot_temperature(ax, df_outside, label="Outside Temp", color="mediumblue")
# plot_temperature(ax, df_weather_forecast, label=f"Weather Forecase - {weather_forecast_update_date}", color="mediumblue")

## The following line is commented out because the data is not reliable
# add_plot(ax, df_inside, label="Inside Temp", color="brown")

fig.suptitle(f"Temperature for {day_obs}")
fig.autofmt_xdate()

ax.grid(":", alpha=0.25)
ax.legend(loc="lower left")
ax.set_title("Temperature at the Outside, Inside, and Glycol Water")
ax.set_xlabel('Time [UTC]')
ax.set_ylabel('Temperature (ºC)')
ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))
y_min, ymax = ax.get_ylim()
ax.set_ylim(max(0, y_min), ymax)

## Get evaluation data
risk_label = {0:'Low', 1:'Warning', 2:'Critical'}
time_plot, temp_diff, rate_change, risk = evaluate_data(df_m1m3, df_outside)
ax2 = fig.add_subplot(gs[1])
ax2.grid(":", alpha=0.25)
ax2.plot(time_plot, temp_diff, label='Temp. Diff')
ax2.plot(time_plot, rate_change, label='Rate change')
ax2.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))
ax2.set_xlabel('Time [UTC]')
ax2.set_ylabel('Temperature Difference ($\Delta$ºC)')
ax2.set_title(f"Temperature Difference between M1M3 and Outside: Risk is {risk_label[risk]}", fontsize=20)
ax2.legend()

plt.savefig(f"./plots/temperature_outside_{day_obs}.png")
plt.show()


## Hourly statistics

Here we have a table per topic with the min/mean/max value every hour. 

In [ ]:
def print_dataframe(df):
    """
    Print the first and last 5 rows of the dataframe.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe to print.
    """
    datetime_format = '%Y-%m-%d %H:%M UTC'

    # Resample each data frame to hourly frequencay
    df = df.resample('H').nearest()

    # Change the format of the index to include only year, month, day, hour, and minute
    df.index = df.index.strftime(datetime_format)

    # Round the temperature values to two decimal places
    df = df.round(2)

    print(df)

In [ ]:
print_dataframe(df_outside)

In [ ]:
print_dataframe(df_m1m3)

In [ ]:
print_dataframe(df_inside_dome)

In [ ]:
print_dataframe(df_m1m3ts_inside.mean(axis=1))

In [ ]:
print_dataframe(df_m1m3ts_glycol_loop_1)

In [ ]:
print_dataframe(df_m1m3ts_glycol_loop_2)

In [ ]:
print_dataframe(df_m1m3ts_glycol_loop_3)